In [1]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs
import tensorflow as tf

In [2]:
# test case list 
# test_case 0 -> Use CT Data to detect cancer (Keras Neural Network)
# test_case 1 -> use CT_Data + Clinic Data to predict cancer (Keras Neural Network)
# test_case 2 -> Use CT Data to detect all 11 disease (Keras Neural Network)
# test_case 3 -> Use CT Data + Clinic Data to detect all 11 disease (Keras Neural Network)

test_case = [0,1,2,3]
n_split = 8
re_train_case_0 = False
re_train_case_1 = False
re_train_case_2 = False
re_train_case_3 = False



In [3]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

# Test Case 0 - Keras CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. This test run uses a neural network constructed using Keras

In [4]:
with tf.device("cpu:0"):
    if (0 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        X = np.array(CT_Data)
        X = X.T
        X = X.astype(float)
        y = np.array(Outcome_Data[21])
        y = y.astype(int)
        standardScalar = StandardScaler()
        standardScalar.fit(X)
        standardScalar.transform(X)

        if re_train_case_0 == True:
            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=11, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(1, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=3000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case0")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case0", custom_objects=None, compile=True, options=None)


        # =========================================================
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score
        from sklearn.metrics import f1_score
        test_y = model.predict(X)
        print(X.shape)
        print(test_y.shape)
        test_y = test_y > 0.5
        right_count = 0
        wrong_count = 0
        cancer_dectected = 0 
        total_cancer = 0
        for i in range(len(test_y)):
            if y[i] != 0:
                total_cancer = total_cancer + 1


            if test_y[i] == y[i]:
                right_count = right_count + 1
                if test_y[i] == True:
                    cancer_dectected = cancer_dectected + 1
            else:
                wrong_count = wrong_count + 1
        print(right_count)
        print(wrong_count)
        print(cancer_dectected)
        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected) + " Cancer detection rate: "+str(cancer_dectected/total_cancer))
        print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
        print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
        print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

2022-05-11 14:11:11.053601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 14:11:11.053918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 14:11:11.080648: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 14:11:11.080990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 14:11:11.081169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

(9223, 11)
(9223, 1)
8979
244
1057
validation accuracy is: 0.9735443998698905. Successfully detected cancer: 1057 Cancer detection rate: 0.8742762613730356
precision score is: 91.9930%
recall_score is: 87.4276%
f1_score: 89.6522%


# Test Case 1 - NN CT+Clinic-To-Cancer Prediction
In this test run we tried to use CT_Data + Clinic Data to predict cancer

In [5]:
with tf.device("cpu:0"):
    if (1 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        X1 = np.array(CT_Data)
        X1 = X1.T
        X1 = X1.astype(float)
        print(X1.shape)

        X2 = np.array(Clininc_Data)
        X2 = X2[4:14,:]
        X2 = X2.T
        X2 = X2.astype(float)
        print(X2.shape)
        
        X = np.column_stack((X1,X2))

        print(X.shape)



        y = np.array(Outcome_Data[21])
        y = y.astype(int)
        standardScalar = StandardScaler()
        standardScalar.fit(X)
        standardScalar.transform(X)

        if re_train_case_1 == True:
            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=21, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(1, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=3000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case1")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case1", custom_objects=None, compile=True, options=None)


        # =========================================================
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score
        from sklearn.metrics import f1_score
        test_y = model.predict(X)
        print(X.shape)
        print(test_y.shape)
        test_y = test_y > 0.5
        right_count = 0
        wrong_count = 0
        cancer_dectected = 0
        total_cancer = 0
        for i in range(len(test_y)):
            if y[i] != 0:
                total_cancer = total_cancer + 1

            if test_y[i] == y[i]:
                right_count = right_count + 1
                if test_y[i] == True:
                    cancer_dectected = cancer_dectected + 1
            else:
                wrong_count = wrong_count + 1
        print(right_count)
        print(wrong_count)
        print(cancer_dectected)
        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected) +" cacner detection rate: "+str(cancer_dectected/total_cancer) )
        print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
        print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
        print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

(9223, 11)
(9223, 10)
(9223, 21)
(9223, 21)
(9223, 1)
9043
180
1121
validation accuracy is: 0.9804835736745093. Successfully detected cancer: 1121 cacner detection rate: 0.9272125723738627
precision score is: 92.4155%
recall_score is: 92.7213%
f1_score: 92.5681%


# Test Case 2 - Keras CT-To-AllOutcome Prediction
This is a test run perform prediction of all outcomes using (CT Data Only). This test run uses a neural network constructed using Keras

In [6]:
with tf.device("cpu:0"):
    if (2 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        X = np.array(CT_Data)
        X = X.T
        X = X.astype(float)

        y = np.array(Outcome_Data)
        y = y.T
        y = y.astype(int)

        # the predictions are:
        # Idx 1 -> CVD
        # Idx 3 -> Heart Failure
        # Idx 5 -> MI DX
        # Idx 7 -> Type 2 Diabetes
        # Idx 9 -> Femoral Neck Fracture
        # Idx 11 -> Unspec Femoral Fracture
        # Idx 13 -> Forearm Fracture
        # Idx 15 -> Humerus Fracture
        # Idx 17 -> Pathologic Fracture
        # Idx 19 -> Alzheimers
        # Idx 21 -> Cancer (weather the patience has or doesn't have, regardless of 1 or 2 types)
        y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]

        if re_train_case_2 == True:


            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=11, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(11, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=3000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case2")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case2", custom_objects=None, compile=True, options=None)
            print(model.evaluate(X, y))  

        y_predict = model.predict(X)
        y_predict = y_predict > 0.5

        total_disease = 0
        count_right = 0
        count_wrong = 0
        detected_disease = 0
        detected_wrong = 0
        for i in range((y_predict.shape)[0]):
            for j in range((y_predict.shape)[1]):
                if y[i,j] == y_predict[i,j]:
                    count_right = count_right + 1
                    if y[i,j] !=0:
                        detected_disease = detected_disease + 1
                else:
                    if y[i,j] ==0:
                        detected_wrong = detected_wrong + 1
                    count_wrong = count_wrong + 1
                
                if y[i,j] != 0:
                    total_disease = total_disease + 1


        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected disease: "+str(detected_disease) +". disease detection rate: "+str(detected_disease/total_disease), ". wrongly diagnosed disease: " + str(detected_wrong) )




289/289 [==============================] - 0s 544us/step - loss: 0.0875 - accuracy: 0.3989 - precision: 0.8587 - recall: 0.8207
[0.08753739297389984, 0.3988940715789795, 0.8586598038673401, 0.820737898349762]
validation accuracy is: 0.9804835736745093. Successfully detected disease: 5984. disease detection rate: 0.8207378960362091 . wrongly diagnosed disease: 985


# Test Case 3 - Keras CT+Clinic-To-AllOutcome Prediction
This is a test run perform prediction of all outcomes using CT Data + Clinic Data. This test run uses a neural network constructed using Keras

In [7]:
with tf.device("cpu:0"):
    if (3 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        X1 = np.array(CT_Data)
        X1 = X1.T
        X1 = X1.astype(float)
        print(X1.shape)

        X2 = np.array(Clininc_Data)
        X2 = X2[4:14,:]
        X2 = X2.T
        X2 = X2.astype(float)
        print(X2.shape)
        
        X = np.column_stack((X1,X2))

        print(X.shape)

        y = np.array(Outcome_Data)
        y = y.T
        y = y.astype(int)

        # the predictions are:
        # Idx 1 -> CVD
        # Idx 3 -> Heart Failure
        # Idx 5 -> MI DX
        # Idx 7 -> Type 2 Diabetes
        # Idx 9 -> Femoral Neck Fracture
        # Idx 11 -> Unspec Femoral Fracture
        # Idx 13 -> Forearm Fracture
        # Idx 15 -> Humerus Fracture
        # Idx 17 -> Pathologic Fracture
        # Idx 19 -> Alzheimers
        # Idx 21 -> Cancer (weather the patience has or doesn't have, regardless of 1 or 2 types)
        y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]

        if re_train_case_3 == True:
            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=21, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(11, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=3000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case3")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case3", custom_objects=None, compile=True, options=None)
            print(model.evaluate(X, y))  



        y_predict = model.predict(X)
        y_predict = y_predict > 0.5

        total_disease = 0
        count_right = 0
        count_wrong = 0
        detected_disease = 0
        detected_wrong = 0
        for i in range((y_predict.shape)[0]):
            for j in range((y_predict.shape)[1]):
                if y[i,j] == y_predict[i,j]:
                    count_right = count_right + 1
                    if y[i,j] !=0:
                        detected_disease = detected_disease + 1
                else:
                    if y[i,j] ==0:
                        detected_wrong = detected_wrong + 1
                    count_wrong = count_wrong + 1
                
                if y[i,j] != 0:
                    total_disease = total_disease + 1


        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected disease: "+str(detected_disease) +". disease detection rate: "+str(detected_disease/total_disease), ". wrongly diagnosed disease: " + str(detected_wrong) )

(9223, 11)
(9223, 10)
(9223, 21)
289/289 [==============================] - 0s 557us/step - loss: 0.0995 - accuracy: 0.3722 - precision: 0.8458 - recall: 0.8179
[0.09950190037488937, 0.37222161889076233, 0.8458155989646912, 0.8178576231002808]
validation accuracy is: 0.9804835736745093. Successfully detected disease: 5963. disease detection rate: 0.8178576326978466 . wrongly diagnosed disease: 1087
